In [ ]:
%load_ext autoreload
%autoreload 2

from pydantic import BaseModel
from encourage.llm import BatchInferenceRunner
from encourage.prompts.prompt_collection import PromptCollection
from encourage.llm import SamplingParams
from encourage.prompts.context import Context
from encourage.prompts.meta_data import MetaData
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Define the toy data structure

In [14]:
sys_prompts = "You are a helpful AI."

# List of user prompts (questions or requests for the AI)
user_prompts = ["What is the capital of France?", "What is the capital of Germany?"] * 5

# # Context information for each prompt (additional data or background info)
contexts = [Context.from_prompt_vars({"key1": "value1"}), Context.from_prompt_vars({"key2": "value2"})] * 5

# # Metadata associated with each prompt (e.g., priority, tags)
meta_datas = [MetaData({"meta": "data1"}), MetaData({"meta": "data2"})] * 5

# Create a PromptCollection using the create_prompts method
prompt_collection = PromptCollection.create_prompts(
    sys_prompts=sys_prompts,  # System prompt or list of system prompts
    user_prompts=user_prompts,  # List of user prompts
    contexts=contexts,  # List of Context objects
    meta_datas=meta_datas,  # List of MetaData objects
)


### Init the Inference Runner with no structured output

In [15]:
sampling_params = SamplingParams(temperature=0.5, max_tokens=100)
runner = BatchInferenceRunner(sampling_params, model_name)
responses = runner.run(prompt_collection)
responses.print_response_summary()

--------------------------------------------------
🧑‍💻 User Prompt:
What is the capital of France?

💬 Response:
The capital of France is Paris.

🤖 System Prompt:
You are a helpful AI.

🗂️ Metadata: MetaData(tags={'meta': 'data1'})
🆔 Request ID: chatcmpl-17fdf391ee7b40f0832dd73c43487f10
🆔 Prompt ID: f233b108-53cf-4d8b-8ded-eef3ba2a8176
🆔 Conversation ID: 0
⏳ Processing Time: -1737124120.0 seconds

--------------------------------------------------
🧑‍💻 User Prompt:
What is the capital of Germany?

💬 Response:
The capital of Germany is Berlin.

🤖 System Prompt:
You are a helpful AI.

🗂️ Metadata: MetaData(tags={'meta': 'data2'})
🆔 Request ID: chatcmpl-1d84dfaebf1b489f8edda45a4691958e
🆔 Prompt ID: dfb1a7be-e980-4b28-abc1-56da684cd276
🆔 Conversation ID: 0
⏳ Processing Time: -1737124120.0 seconds

--------------------------------------------------
🧑‍💻 User Prompt:
What is the capital of France?

💬 Response:
The capital of France is Paris.

🤖 System Prompt:
You are a helpful AI.

🗂️ Metadata:

### Change the user request and add a structured output with pydantic model

In [10]:
user_prompts = ["Return a male User", "Return a female User" ] * 5
prompt_collection = PromptCollection.create_prompts(
    sys_prompts=sys_prompts, 
    user_prompts=user_prompts, 
    contexts=contexts, 
    meta_datas=meta_datas
)

class User(BaseModel):
    name: str
    age: int
    id: str
    


sampling_params = SamplingParams(temperature=0.5, max_tokens=1000)

runner = BatchInferenceRunner(sampling_params, model_name)
responses = runner.run(prompt_collection, User)
responses.print_response_summary()

--------------------------------------------------
🧑‍💻 User Prompt:
Return a male User

💬 Response:
{"name": "John Doe", "age": 30, "id": "user_001"}

🤖 System Prompt:
You are a helpful AI.

🗂️ Metadata: MetaData(tags={'meta': 'data1'})
🆔 Request ID: chatcmpl-64b508e613b045cb85ee627f7105d5f7
🆔 Prompt ID: cb5f0e8b-6616-4213-b993-4ec1550d3634
🆔 Conversation ID: 0
⏳ Processing Time: -1737123949.0 seconds

--------------------------------------------------
🧑‍💻 User Prompt:
Return a female User

💬 Response:
{ "name": "Emily Wilson", "age": 28, "id": "user_123" }

🤖 System Prompt:
You are a helpful AI.

🗂️ Metadata: MetaData(tags={'meta': 'data2'})
🆔 Request ID: chatcmpl-8f3be11169894f3db329d0c958a327b9
🆔 Prompt ID: a0f26182-f8af-48fa-951f-c3bb70f80ba1
🆔 Conversation ID: 0
⏳ Processing Time: -1737123951.0 seconds

--------------------------------------------------
🧑‍💻 User Prompt:
Return a male User

💬 Response:
{"name": "John Doe", "age": 30, "id": "user_001"}

🤖 System Prompt:
You are a he

### Change the user request and add a structured output with custom json model

In [12]:

user_prompts = ["Return a male User", "Return a female User" ] * 5
prompt_collection = PromptCollection.create_prompts(
    sys_prompts=sys_prompts, 
    user_prompts=user_prompts, 
    contexts=contexts, 
    meta_datas=meta_datas, 
)

schema = """
{
  "title": "User",
  "type": "object",
  "properties": {
    "name": {"type": "string"},
    "last_name": {"type": "string"},
    "id": {"type": "integer"},
    "height": {"type": "integer"}
  },
  "required": ["name", "last_name", "id", "height"]
}
"""
    

sampling_params = SamplingParams(temperature=0.5, max_tokens=1000)

runner = BatchInferenceRunner(sampling_params, model_name)
responses = runner.run(prompt_collection, schema)
responses.print_response_summary()

--------------------------------------------------
🧑‍💻 User Prompt:
Return a male User

💬 Response:
{"name": "John Doe", "last_name": "Doe", "id": 1, "height": 180}

🤖 System Prompt:
You are a helpful AI.

🗂️ Metadata: MetaData(tags={'meta': 'data1'})
🆔 Request ID: chatcmpl-0165e1527c8c419fa75f795e232ebfee
🆔 Prompt ID: d8fc8192-6cab-4340-8a52-a4d49dd896f3
🆔 Conversation ID: 0
⏳ Processing Time: -1737124089.0 seconds

--------------------------------------------------
🧑‍💻 User Prompt:
Return a female User

💬 Response:
{ "name": "Emily Johnson", "last_name": "Johnson", "id": 12345, "height": 5 }

🤖 System Prompt:
You are a helpful AI.

🗂️ Metadata: MetaData(tags={'meta': 'data2'})
🆔 Request ID: chatcmpl-7dbc7a2897274ed88076df2081f578f1
🆔 Prompt ID: 6ad6415b-744f-4cc3-a528-52d92cd505a0
🆔 Conversation ID: 0
⏳ Processing Time: -1737124089.0 seconds

--------------------------------------------------
🧑‍💻 User Prompt:
Return a male User

💬 Response:
{"name": "John Doe", "last_name": "Doe", "i